# dump-to-db
Dump generated glyphs to MongoDB

## set up connection

In [1]:
import sys

In [2]:
sys.path.append('/usr/lib/python3/dist-packages/')

In [3]:
%cd ..

import os
import sys
import pymongo
import fonts.font_processing as font_processing

/mnt/d/grad23/fonty


In [4]:
sys.path

['/mnt/d/grad23/fonty/notebooks',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/root/.local/share/virtualenvs/fonty-CHIKu2Gx/lib/python3.10/site-packages',
 '/usr/lib/python3/dist-packages/']

In [5]:
cli_remote = pymongo.MongoClient(
    'mongodb+srv://fonty.hquocfa.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority',
    tls=True,
    tlsCertificateKeyFile='/mnt/d/grad23/fonty/auth/X509-cert-6658948524510096383.pem'
)

In [6]:
cli_remote

MongoClient(host=['ac-5dsaw84-shard-00-00.hquocfa.mongodb.net:27017', 'ac-5dsaw84-shard-00-01.hquocfa.mongodb.net:27017', 'ac-5dsaw84-shard-00-02.hquocfa.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='$external', authmechanism='MONGODB-X509', retrywrites=True, w='majority', replicaset='atlas-32tpky-shard-0', tls=True, tlscertificatekeyfile='/mnt/d/grad23/fonty/auth/X509-cert-6658948524510096383.pem')

## collect fonts

In [7]:
fonts = [
    '/mnt/d/grad23/font-times-new-roman/times-b.ttf',
    '/mnt/d/grad23/font-times-new-roman/times-bi.ttf',
    '/mnt/d/grad23/font-times-new-roman/times-i.ttf',
    '/mnt/d/grad23/font-times-new-roman/times.ttf'
]

### convert

In [8]:
font_processors = [
    font_processing.FontProcessor(faddr)
    for faddr in fonts
]

The following table(s) in the font have been ignored by FontForge
  Ignoring 'DSIG' digital signature table
  Ignoring 'LTSH' linear threshold table
  Ignoring 'PCLT' PCL 5 data table
  Ignoring 'VDMX' vertical device metrics table
  Ignoring 'hdmx' horizontal device metrics table
Glyph 1295 is called ".notdef", a singularly inept choice of name (only glyph 0
 may be called .notdef)
FontForge will rename it.
The glyph named macron is mapped to U+02C9.
But its name indicates it should be mapped to U+00AF.
The glyph named Delta is mapped to U+0394.
But its name indicates it should be mapped to U+2206.
The glyph named Omega is mapped to U+03A9.
But its name indicates it should be mapped to U+2126.
The glyph named mu is mapped to U+03BC.
But its name indicates it should be mapped to U+00B5.
The glyph named finalkafsheva is mapped to U+E802.
But its name indicates it should be mapped to U+05DA.
The glyph named finalkafqamats is mapped to U+E803.
But its name indicates it should be mapped to

In [9]:
glyphsets = [
    processor.glyphset_from_unicode_subset(
        font_processing.get_alphabet('all')
    )
    for processor in font_processors
]

In [10]:
jsons = [
    glyphset.to_json()
    for glyphset in glyphsets
]

In [11]:
jsons[0]

{'panose': (2, 2, 8, 3, 7, 5, 5, 2, 3, 4),
 'unicoderanges': (31367, -2147483648, 8, 0),
 'weight': 700,
 'cap_height': 1356.0,
 'family_name': 'Times New Roman',
 'font_name': 'Times New Roman Bold',
 'glyphs': [{'letter': '!',
   'path': 'M364 441h-44q-7 114 -46 276l-52 218q-47 196 -47 281q0 77 46 124t120 47q72 0 119.5 -47.5t47.5 -120.5q0 -76 -50 -284l-53 -218q-28 -116 -41 -276zM343 298q67 0 114.5 -47.5t47.5 -115.5q0 -67 -47.5 -114.5t-114.5 -47.5t-114.5 47.5t-47.5 114.5t47 115t115 48z '},
  {'letter': '"',
   'path': 'M349 721h-49l-104 385q-20 75 -20 132q0 62 43.5 105.5t104.5 43.5q70 0 109 -45.5t39 -98.5q0 -45 -24 -139zM832 721h-50l-101 384q-24 88 -24 136q0 58 44.5 102t106.5 44q71 0 108.5 -47t37.5 -96q0 -35 -26 -139z'},
  {'letter': '#',
   'path': 'M90 -28l86 424h-136v133h162l59 292h-221v130h247l87 433h132l-88 -433h295l86 433h132l-88 -433h140v-130h-165l-60 -292h225v-133h-250l-86 -424h-133l86 424h-291l-90 -424h-129zM335 529h294l58 292h-294z'},
  {'letter': '$',
   'path': 'M566 835q2

In [12]:
target_collection = cli_remote['fonty']['fonts']
target_collection

Collection(Database(MongoClient(host=['ac-5dsaw84-shard-00-00.hquocfa.mongodb.net:27017', 'ac-5dsaw84-shard-00-01.hquocfa.mongodb.net:27017', 'ac-5dsaw84-shard-00-02.hquocfa.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='$external', authmechanism='MONGODB-X509', retrywrites=True, w='majority', replicaset='atlas-32tpky-shard-0', tls=True, tlscertificatekeyfile='/mnt/d/grad23/fonty/auth/X509-cert-6658948524510096383.pem'), 'fonty'), 'fonts')

In [16]:
target_collection.delete_many({'family_name': {'$regex': r'.*Times.*'}})

In [17]:
for font in jsons:
    target_collection.insert_one(font)